In [8]:
import torch.nn.functional as F
from timeit import default_timer
from utilities3 import *
from matplotlib import pyplot as plt
from pdb import set_trace as st
import cv2
from unet_parts import *
from pytorch_msssim import  ms_ssim, ssim
from metrics import *
import wandb
import sys
import json
import resnet

class Dataset(torch.utils.data.Dataset):
    def __init__(self, train_F,  transform=None, mean=None, std=None,  eps=1e-10):
        'Initialization'
        with open(train_F, 'r') as openfile: 
            data = json.load(openfile) 
        data = np.array(data)
        #self.labels = data[:,1].astype(int)
        self.data = data[:,0]
        self.targets = data[:,1].astype(float)
        self.mean = mean
        self.std = std
        self.eps = eps

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.data)

    def __getitem__(self, index):
        x = torch.load(self.data[index])
        try:
            x = (x - self.mean) / (self.std + self.eps)
        except:
            print(x.shape)
        return x, self.targets[index]

torch.manual_seed(0)
np.random.seed(0)

N = 700 * 3
ntrain = 1760 #900*2
ntest = 2200-1760#150*2

batch_size = 4
learning_rate = 1e-2#0.001
epochs = 90
iterations = epochs * (ntrain // batch_size)

modes = 20 #12
modes_lat = 12#12
width = 16 #28 #32

r = 10
x_lat = 32#64
x_dep = 955#857
y_lat = 877
y_dep = 844
x_dep_sample = 2

#noise_level = '20dB'
USE_WANDB = True
data_dir1 = '/mlroom/data/lung_set1' #pts/'
data_dir = '/mlroom/data/lung_set1_fix'
EVALUATE = 1-True

mean_file = torch.load('raw_mean.pt')
std_file = torch.load('raw_std.pt')

train_set = Dataset('add_a0_4500_train.json', mean = mean_file, std=std_file)
test_set = Dataset('add_a0_4500_test.json', mean = mean_file, std=std_file)

train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size,
                                            shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size,
                                            shuffle=True)#False)
ntrain = len(train_loader)
ntest = len(test_loader)

In [9]:
model = resnet.resnet18(num_classes=1).cuda()

In [ ]:
for raw, aera in train_set:
    print(aera)